In [3]:
!g++ -march=native -Q --help=target

The following options are target specific:
  -m128bit-long-double        		[enabled]
  -m16                        		[disabled]
  -m32                        		[disabled]
  -m3dnow                     		[disabled]
  -m3dnowa                    		[disabled]
  -m64                        		[enabled]
  -m80387                     		[enabled]
  -m8bit-idiv                 		[disabled]
  -m96bit-long-double         		[disabled]
  -mabi=                      		sysv
  -mabm                       		[enabled]
  -maccumulate-outgoing-args  		[disabled]
  -maddress-mode=             		long
  -madx                       		[enabled]
  -maes                       		[enabled]
  -malign-data=               		compat
  -malign-double              		[disabled]
  -malign-functions=          		0
  -malign-jumps=              		0
  -malign-loops=              		0
  -malign-stringops           		[enabled]
  -mandroid                   		[disabled]
  -march=                     		cascadelake
  -masm=         

In [2]:
# https://github.com/Lgeu/hungry_geese/commit/0b0eda7e1a4b6d2d5f5a083a4e040b84e8eb9725
!g++ ../../../KKT89/Combine.cpp -O3 -Ofast -march=native -std=c++17

In file included from ../../../KKT89/src/Goose.hpp:3,
                 from ../../../KKT89/src/Stage.hpp:4,
                 from ../../../KKT89/src/Duct.hpp:6,
                 from ../../../KKT89/src/Duct.cpp:2,
                 from ../../../KKT89/Combine.cpp:3:
../../../KKT89/src/Evaluator.hpp: In member function ‘void hungry_geese::evaluation_function::BitBoard::ShiftRight()’:
../../../KKT89/src/library.hpp:64:9: warning: format ‘%d’ expects argument of type ‘int’, but argument 3 has type ‘hungry_geese::evaluation_function::BitBoard::ull’ {aka ‘long long unsigned int’} [-Wformat=]
   64 |         "`%s` (%d) is out of range [%d, %d)", #value, value, left, right)
      |         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
../../../KKT89/src/library.hpp:54:24: note: in definition of macro ‘ASSERT’
   54 |                 printf(__VA_ARGS__); \
      |                        ^~~~~~~~~~~
../../../KKT89/src/Evaluator.hpp:664:9: note: in expansion of macro ‘ASSERT_RANGE’
  664 |         ASSERT_

In file included from ../../../KKT89/src/Duct.hpp:9,
                 from ../../../KKT89/src/Duct.cpp:2,
                 from ../../../KKT89/Combine.cpp:3:
../../../KKT89/src/Evaluator.hpp: In member function ‘void hungry_geese::evaluation_function::Model<in_dim, out_dim, hidden_1, hidden_2>::LoadParameters(const char*) [with int in_dim = 2382; int out_dim = 5; int hidden_1 = 256; int hidden_2 = 32]’:
../../../KKT89/src/Evaluator.hpp:492:14: warning: ignoring return value of ‘size_t fread(void*, size_t, size_t, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
  492 |         fread(&embed.parameters.weight, embed.parameters.weight.Ravel().size(), sizeof(short), fp);
      |         ~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
../../../KKT89/src/Evaluator.hpp:493:14: warning: ignoring return value of ‘size_t fread(void*, size_t, size_t, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
  493 |       

In [4]:
!mkdir 1vs3
!mkdir 2vs2
!mkdir 3vs1

In [7]:
ls ../../rl/parameters

013_01.bin


In [8]:
# コマンド確認
param1 = "../../../KKT89/src/param_010_01.bin"
param2 = "../../rl/parameters/013_01.bin"
!./a.out -j -o ./ -t 0.1 -p {param1} {param2} {param1} {param2}

In [28]:
from subprocess import Popen
N_MATCHES = 400
p1 = Popen(f"seq 1 {N_MATCHES} | xargs -I@ -P 20 sh -c './a.out -j -o ./1vs3/ -t 0.1 -p {param1} {param2} {param2} {param2}'", shell=True)
p2 = Popen(f"seq 1 {N_MATCHES} | xargs -I@ -P 20 sh -c './a.out -j -o ./2vs2/ -t 0.1 -p {param1} {param1} {param2} {param2}'", shell=True)
p3 = Popen(f"seq 1 {N_MATCHES} | xargs -I@ -P 20 sh -c './a.out -j -o ./3vs1/ -t 0.1 -p {param1} {param1} {param1} {param2}'", shell=True)

In [42]:
!ls -1 1vs3 | wc -l
!ls -1 2vs2 | wc -l
!ls -1 3vs1 | wc -l

500
500
500


In [14]:
!zip -r -q 010_vs_013.zip 1vs3 2vs2 3vs1

In [43]:
import numpy as np

ranks_param2 = []
files = !ls -1 3vs1
for file in files:
    ranks = !cat ./3vs1/{file} | tail -n 1
    ranks = list(map(int, ranks[0].split()))
    ranks_param2.append(ranks[-1])

np.bincount(ranks_param2)

array([  0,  93,  91, 136, 180])

In [44]:
ranks_param2 = []
files = !ls -1 2vs2
for file in files:
    ranks = !cat ./2vs2/{file} | tail -n 1
    ranks = list(map(int, ranks[0].split()))
    ranks_param2.append(ranks[2])
    ranks_param2.append(ranks[3])
np.bincount(ranks_param2)

array([  0, 241, 225, 247, 287])

In [45]:
ranks_param2 = []
files = !ls -1 1vs3
for file in files:
    ranks = !cat ./1vs3/{file} | tail -n 1
    ranks = list(map(int, ranks[0].split()))
    ranks_param2.append(ranks[1])
    ranks_param2.append(ranks[2])
    ranks_param2.append(ranks[3])
np.bincount(ranks_param2)

array([  0, 364, 378, 381, 377])